In [1]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import math
from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
import matplotlib.gridspec as gridspec
from IPython.display import Markdown, display, HTML
from time import gmtime, strftime
import pickle
import analyzetool
import analyzetool.gas as gasAnalyze
import analyzetool.liquid as liqAnalyze
import warnings
warnings.filterwarnings('ignore') # make the notebook nicer

%matplotlib inline

display(HTML("<style>.rendered_html { font-size: 20px; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

KB_J = 1.38064852e-23 #J/K
E0 = 8.854187817620e-12
N_vis = 8.91e-4 #Pa.s
R=1.9872036E-3
NA=6.02214129*(1e23)

dref = 997.045
hvref = 43.989/4.184
alpharef = 2.572
kapparef = 45.247
dielref = 78.409
diffref = 2.23
angref = 106.1

def diff_correction(box_size,T):
    const = (2.837297)/(6*np.pi)
    corr = (1e4)*const*KB_J*T/(N_vis*box_size*(1.0E-10))
    return corr

def devcalc(a,ref):
    return 100*(a-ref)/ref

In [7]:
base_dir = "/user/roseane/HIPPO/water_param/hippo_tinker9"
sim_dir = f"{base_dir}/simulations/init_params"
#sim_dir = "/work/roseane/HIPPO/water_param/forcebalance/cf-trial-1/fit.tmp/Liquid/iter_0000/298.15K-1.0atm"
ref_dir = "/user/roseane/HIPPO/water_param/reference_data"
dir_gr = f"{ref_dir}/g_r_data/"

sim_dir = "/user/roseane/HIPPO/water_param/hippo_tinker9/simulations/init_params"
sim_dir = "/work/roseane/HIPPO/water_param/hippo_tinker9/simulations/t18-50A-NPT"
#sim_dir = "/work/roseane/HIPPO/water_param/forcebalance/test_iter/trial18-NPT"
res_dir = f'{sim_dir}/results'


save_dir = f"{sim_dir}/results"
os.system(f"mkdir -p {save_dir}")
save = True

t = 298.15
tsav = 298
sim_path = f"{sim_dir}/sim_{tsav}"
dirnm = sim_path.split('/')[-2]

liquid = liqAnalyze.Liquid(sim_path,xyzfile='liquid.xyz',n_atoms_mol=3,temperature=t,equil=2000,
                         logfile='liquid.log',analyzelog='analysis.log')
#                          logfile=None,analyzelog='analysis.log')

gas_dir = "/work/roseane/HIPPO/water_param/hippo_tinker9/simulations/t18-50A-NVT"
gas_dir = "/work/roseane/HIPPO/water_param/hippo_tinker9/simulations/t18-50A-NVT/gas-simulations"
#gas_dir=sim_dir
liquid.all_properties(f'{gas_dir}/sim_{tsav}/gas.log',f'{sim_path}/analysis.log')
liquid.get_diffusion(f'{sim_path}/diffusion.log')


gOO = liquid.get_g_r(f"{sim_path}/g_OO_r.log")
gOH = liquid.get_g_r(f"{sim_path}/g_OH_r.log")
gHH = liquid.get_g_r(f"{sim_path}/g_HH_r.log")

#ref g_r
g_OO_exp = np.load(f"{dir_gr}/g_OO_exp.npy")
g_OO2 = np.load(f"{dir_gr}/g_OO2.npy")
g_OO_APS2013 = np.load(f"{dir_gr}/g_OO_APS2013.npy")
g_OO_APS2014 = np.load(f"{dir_gr}/g_OO_APS2014.npy")
g_OO_s1986 = np.load(f"{dir_gr}/g_OO_s1986.npy")
g_OO_s2014 = np.load(f"{dir_gr}/g_OO_s2014.npy")

        
res_data = """
### Average properties for %s (298.15 K - 1 atm)
Density      %7.3f
H

""" % (dirnm,liquid.median_diffusion)
print("%s @ %.2f K" % (dirnm,t))
print("Avg. Density: %5.2f kg/m^3     (%5.2f)" % (liquid.avgRho*1000,devcalc(liquid.avgRho*1000,dref)))
print("Avg. PE/mol : %5.2f kcal/mol" % (liquid.PEmol))
print("Heat Capac. : %5.2f cal/mol/K" % (liquid.Cp))
print("Isot.Compr. : %5.2f 10^-6 bar    (%5.2f)" % (liquid.kappa,devcalc(liquid.kappa,kapparef)))
print("T.Exp.Coef. : %5.2f 10^-4 1/K    (%5.1f)" % ((1e4)*liquid.alpha,devcalc((1e4)*liquid.alpha,alpharef)))
print("Dielectric  : %5.2f              (%5.2f)" % (liquid.dielectric,devcalc(liquid.dielectric,dielref)))
print("Self-diff.  : %5.2f 10^-5 cm^2/s (%5.2f)" % (liquid.median_diffusion,devcalc(liquid.median_diffusion,diffref)))
print("Heat Vapor. : %5.2f Kcal/mol     (%5.2f)" % (liquid.HV,devcalc(liquid.HV,hvref)))
print("Gas Avg.PE. : %5.2f Kcal/mol" % (liquid.gasAvgPE))

#Avg Angle
# liquid.get_coordinates('%s/liquid.arc' % sim_path)
# liquid.compute_avg_angle()
# langle = liquid.avg_angle
# print("Avg. Angle  : %5.2f deg         (%5.2f)" % (langle,devcalc(langle,angref)))

(3, 3)

array([1, 4, 7])